$$ ITI \space AI-Pro: \space Intake \space 45 $$
$$ Recommender \space Systems $$
$$ Lab \space no. \space 2 $$

# `01` Import Necessary Libraries

## `i` Default Libraries

In [2]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2461538 sha256=fec56047ff3847bd693226fc2d21eee074a9801d55de9dcc327ed9b54b058d88
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [5]:
!pip install scikit-surprise --no-binary :all:

In [6]:
from surprise.reader import Reader
from surprise.dataset import Dataset
from surprise.model_selection import train_test_split

In [7]:
import numpy as np
import pandas as pd

## `ii` Additional Libraries
Add imports for additional libraries you used throughout the notebook

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

----------------------------

# `02` Load Data

 The dataset will have the following columns :
   - song_id (String) : Unique identified for the song
   - user_id (String) : Unique identifier for the user
   - song_genre (Integer) : An integer representing a genre for the song, value is between 1 and 5, indicating that there are 5 unique genres. Each song can only have 1 genre
   - artist_id (String) : Unique identifier for the author of the song
   - n_listen (Integer) : The number of times this user has heard the song (0 -> 15)
   - publish_year (Integer) : The year of song publishing

In [10]:
data = pd.read_csv("songs_data.csv")
data.head()

,song_id,artist_id,song_genre,user_id,n_listen,publish_year
0,537,368,4,2066,13,2002
1,921,107,1,1179,5,2006
2,352,188,1,1468,11,2013
3,853,370,4,460,9,2020
4,479,408,2,1125,3,2020


--------------------------

# `03` Content-based Filtering

Practice for content-based filtering on dummy data

## `i` Feature Engineering/Selection
Construct the item vector representation matrix from the `data` above

In [11]:
item_df = data[['song_id', 'artist_id', 'song_genre', 'publish_year']].drop_duplicates()
item_df.set_index('song_id', inplace=True)
item_df.head()

,artist_id,song_genre,publish_year
song_id,,,
537,368,4,2002
921,107,1,2006
352,188,1,2013
853,370,4,2020
479,408,2,2020


## `ii` Utility Matrix
Construct utility matrix for the loaded dataframe `data`

In [12]:
utility_matrix = data.pivot_table(index='user_id', columns='song_id', values='n_listen', fill_value=0)
utility_matrix.head()

song_id,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2,15.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,11.0,0.0,6.0
3,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,11.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0


## `iii` Item-Item Similarity Matrix

Construct item-item (Cosine/Adjusted Cosine) similarity matrix.

In [13]:
encoder = OneHotEncoder(sparse_output=False)
categorical_encoded = encoder.fit_transform(item_df[['artist_id', 'song_genre']])
scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(item_df[['publish_year']])
item_vectors = np.hstack([categorical_encoded, numerical_scaled])
item_sim_matrix = cosine_similarity(item_vectors)
item_similarity_df = pd.DataFrame(item_sim_matrix, index=item_df.index, columns=item_df.index)
item_similarity_df.head()

song_id,537,921,352,853,479,759,146,307,776,137,...,815,324,648,670,657,910,490,319,889,234
song_id,,,,,,,,,,,,,,,,,,,,,
537,1.000000,0.013306,0.026944,0.451320,0.037532,0.028650,0.000000,0.008969,0.011155,0.036210,...,0.037532,0.034827,0.034827,0.004515,0.502257,0.015415,0.026944,0.500849,0.013306,0.034827
921,0.013306,1.000000,0.528378,0.110615,0.110615,0.084440,0.490098,0.026433,0.032877,0.519560,...,0.110615,0.102643,0.102643,0.502296,0.013306,0.522457,0.079409,0.006664,0.519608,0.102643
352,0.026944,0.528378,1.000000,0.223990,0.223990,0.170986,0.458040,0.053526,0.066575,0.601938,...,0.223990,0.207846,0.599555,0.483948,0.026944,0.537818,0.160799,0.013495,0.528378,0.207846
853,0.451320,0.110615,0.223990,1.000000,0.312012,0.238180,0.000000,0.074560,0.092737,0.301026,...,0.312012,0.289524,0.289524,0.037532,0.451320,0.128147,0.223990,0.433288,0.110615,0.289524
479,0.037532,0.110615,0.223990,0.312012,1.000000,0.613313,0.000000,0.074560,0.092737,0.301026,...,0.312012,0.289524,0.644191,0.037532,0.037532,0.128147,0.603911,0.018798,0.110615,0.644191


## `iv` Top-K Candidate Generation

Selet top-K (a k of your choice) similar items for each item (a user of your choice) rated from the similarity matrix above.

In [14]:
K=5
def get_top_k_similar_items(similarity_matrix, k):
    top_k_items = {}
    for idx, row in enumerate(similarity_matrix):
        top_k_indices = np.argsort(row)[-k-1:-1]
        top_k_items[idx] = [(i, row[i]) for i in top_k_indices]
    return top_k_items
top_k_similar = get_top_k_similar_items(item_similarity_df.values, K)

for song_id, similar_items in top_k_similar.items():
    print(f"Top-{K} similar items for song {item_similarity_df.index[song_id]}:")
    for item, sim in similar_items:
        print(f"  - {item_similarity_df.index[item]} (Similarity: {sim:.2f})")
    print()


Top-5 similar items for song 537:
  - 228 (Similarity: 0.50)
  - 878 (Similarity: 0.50)
  - 557 (Similarity: 0.50)
  - 760 (Similarity: 0.50)
  - 474 (Similarity: 0.93)

Top-5 similar items for song 921:
  - 331 (Similarity: 0.53)
  - 667 (Similarity: 0.53)
  - 462 (Similarity: 0.53)
  - 354 (Similarity: 0.53)
  - 682 (Similarity: 0.53)

Top-5 similar items for song 352:
  - 970 (Similarity: 0.61)
  - 397 (Similarity: 0.61)
  - 859 (Similarity: 0.61)
  - 230 (Similarity: 0.98)
  - 536 (Similarity: 0.98)

Top-5 similar items for song 853:
  - 510 (Similarity: 0.66)
  - 336 (Similarity: 0.66)
  - 603 (Similarity: 0.66)
  - 544 (Similarity: 0.66)
  - 605 (Similarity: 1.00)

Top-5 similar items for song 479:
  - 981 (Similarity: 0.66)
  - 633 (Similarity: 0.66)
  - 813 (Similarity: 0.66)
  - 480 (Similarity: 0.66)
  - 358 (Similarity: 0.66)

Top-5 similar items for song 759:
  - 981 (Similarity: 0.62)
  - 802 (Similarity: 0.62)
  - 813 (Similarity: 0.62)
  - 358 (Similarity: 0.62)
  - 597 

## `v` Candidate Filtering

Filter out items (your user) has rated from the candidates above.

In [15]:
rated_items_series = utility_matrix.loc[1179]
rated_items_series = rated_items_series[rated_items_series > 0]

## `vi` Candidate Rating Prediction

Calculate the predicted rating for each of the candidate items.

In [16]:
rated_item_ids = list(rated_items_series.index)
all_item_ids = list(utility_matrix.columns)
candidate_items = [item for item in all_item_ids if item not in rated_item_ids]

predicted_ratings = {}

for candidate in candidate_items:
    numerator = 0
    denominator = 0
    for rated_item in rated_items_series.index:
        if rated_item in item_similarity_df.columns and candidate in item_similarity_df.index:
            sim = item_similarity_df.loc[candidate, rated_item]
            rating = rated_items_series[rated_item]
            numerator += sim * rating
            denominator += abs(sim)

    if denominator > 0:
        predicted_rating = numerator / denominator
        predicted_ratings[candidate] = predicted_rating

for item, rating in sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True):
    print(f"  - Song {item}: {rating:.2f}")

  - Song 854: 9.31
  - Song 146: 9.25
  - Song 162: 9.25
  - Song 254: 9.25
  - Song 327: 9.25
  - Song 591: 9.25
  - Song 645: 9.25
  - Song 920: 9.25
  - Song 944: 9.25
  - Song 161: 9.23
  - Song 620: 9.23
  - Song 367: 9.11
  - Song 503: 9.11
  - Song 577: 9.11
  - Song 773: 9.11
  - Song 841: 9.11
  - Song 881: 9.11
  - Song 126: 8.99
  - Song 206: 8.99
  - Song 384: 8.99
  - Song 482: 8.99
  - Song 582: 8.99
  - Song 670: 8.99
  - Song 700: 8.99
  - Song 948: 8.99
  - Song 53: 8.89
  - Song 243: 8.89
  - Song 392: 8.89
  - Song 404: 8.89
  - Song 595: 8.89
  - Song 989: 8.89
  - Song 901: 8.88
  - Song 156: 8.86
  - Song 590: 8.86
  - Song 564: 8.82
  - Song 12: 8.81
  - Song 118: 8.81
  - Song 134: 8.81
  - Song 251: 8.81
  - Song 315: 8.81
  - Song 318: 8.81
  - Song 360: 8.81
  - Song 602: 8.81
  - Song 646: 8.81
  - Song 743: 8.81
  - Song 470: 8.81
  - Song 11: 8.80
  - Song 168: 8.80
  - Song 347: 8.80
  - Song 504: 8.80
  - Song 728: 8.80
  - Song 823: 8.80
  - Song 916: 8

--------------------------

# `04` KNN Item-based Colaborative Filtering

Practice for Using Scikit Surprise Library

## `i` Data Loading

Load `songsDataset.csv` file into a dataframe

In [20]:
df = pd.read_csv('songsDataset.csv')
df.head()

,userID,songID,rating
0,0,90409,5
1,4,91266,1
2,5,8063,2
3,5,24427,4
4,5,105433,4


## `ii` Prepare Data

Procedures to Follow:
- Instantiate the Reader Object (see, [Documentation](https://surprise.readthedocs.io/en/stable/reader.html))
- Load the Data into `surprise.dataset.Dataset` (see, [Documentation](https://surprise.readthedocs.io/en/stable/dataset.html))
- Build the full (i.e. without folds) `surprise.Trainset` (see, [Documentation](https://surprise.readthedocs.io/en/stable/trainset.html#:~:text=It%20is%20used%20by%20the%20fit()%20method%20of%20every%20prediction%20algorithm.%20You%20should%20not%20try%20to%20build%20such%20an%20object%20on%20your%20own%20but%20rather%20use%20the%20Dataset.folds()%20method%20or%20the%20DatasetAutoFolds.build_full_trainset()%20method.))

In [21]:
reader = Reader(rating_scale=(1, 5))

In [23]:
data = Dataset.load_from_df(df[['userID', 'songID', 'rating']], reader)
data

## `iii` Initialize the `KNNWithMeans` Model

**Note**: `KNNWithMeans` uses the normalized ratings instead of the raw ones. (See [Documentation](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNWithMeans))

**Hint**: Use $k=10$ and configure `sim_options` to be:
- item_based
- pearson

In [24]:
!pip install --upgrade pip setuptools wheel
!pip install numpy==1.24.4
!pip install scikit-surprise --only-binary :all:

  Using cached setuptools-80.4.0-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.0 MB/s eta 0:00:00
Using cached setuptools-80.4.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 146.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.2 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is

In [ ]:
from surprise import KNNWithMeans

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2)
sim_options = {
    'name': 'pearson',
    'user_based': False}
knn_model = KNNWithMeans(k=10, sim_options=sim_options)

## `iv` Fit the Model on Data

In [ ]:
knn_model.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


## `v` Calculate Predicted Rating $\hat{r}$ for User $199988$

**Hine**: you can use `.predict()` method of the model (see [Documentaion](https://surprise.readthedocs.io/en/stable/getting_started.html?highlight=.predict#train-on-a-whole-trainset-and-the-predict-method:~:text=pred%20%3D%20algo.predict(uid%2C%20iid%2C%20r_ui%3D4%2C%20verbose%3DTrue)))

In [ ]:
song_predictions = knn_model.predict(199988)
song_predictions.head()

,songID
0,90409
1,91266
2,8063
3,24427
4,105433


In [ ]:
for index, row in song_predictions.iterrows():
    prediction = knn_model.predict(row['userID'], row['songID'])
    song_predictions.at[index, 'predicted_rating'] = prediction.est
song_predictions.head()

,predicted_rating
songID,
90409,4.808493
91266,4.705610
8063,4.239800
24427,4.549136
105433,4.872347


## `vi` Recommend Top 10 Songs

In [ ]:
song_predictions_sorted = song_predictions.sort_values(by='predicted_rating', ascending=False)
song_predictions_sorted.head(10)

,predicted_rating
songID,
60888,5.000000
122065,5.000000
132189,5.000000
71582,5.000000
52611,5.000000
62954,5.000000
112023,5.000000
40712,5.000000
126757,4.983563


----------------------------------------------

$$ Wish \space you \space all \space the \space best \space ♡ $$
$$ Mahmoud \space Shawqi $$